# Bioinformatics Platform using JupyterLab

Author: Robert Bradford

This is a thesis project using JupyterLab to facilitate the study of BIOC-4010 course. This tutorial implements the following basic modules using Biopython, NGLView。
1. Databases and sequencing file formats
2. Translating DNA sequences into a protein sequence
3. Calculating DNA GC content
4. Pairwise sequence alignments using Needleman and Waterman algorithms
5. Substitution matrices
6. BLAST
7. Display a 3D structure

Requirements:

To run this notebook successfully, it is recommended to use Miniconda + JupyterLab and install the required packages and extensions. The notebook shall also work on [Google Colab](https://colab.research.google.com/) or [Binder](https://jupyter.org/binder) but this has not been tested.

The following packages are required and can be installed using conda. It is recommended to create a new environment and install these packages. You can use the [nglview-jupyterlab.sh script](https://github.com/nglviewer/nglview/blob/master/devtools/nglview-jupyterlab.sh) to install the nglview related packages.
* python 3.8+
* jupyterlab 2.1+
* biopython 1.7+
* ipywidgets 7.5+
* nodejs 12.0.0+, required for the jupyter-labextensions
* nglview 2.7+
    * if you do not use the nglview-jupyterlab.sh script, run the following two commands manually after you install jupyterlab
    * `jupyter-labextension install @jupyter-widgets/jupyterlab-manager`
    * `jupyter-labextension install nglview-js-widgets@$nglviewversion` where `$nglviewversion` is the version of the `nglview` installed package, which can be inspected with `conda list nglview`.
    

This notebook has been test on:
1. miniconda3
2. command line Windows (NGLView generated 3D structures do not load)
3. 

## Setup environment

In [1]:
import Bio
from Bio.Seq import Seq
import ipywidgets as widgets
from Bio.PDB import *
import os
import sys
import nglview as nv
import platform
print("Python version",sys.version_info)
print("Biopython version", Bio.__version__)

Python version sys.version_info(major=3, minor=9, micro=1, releaselevel='final', serial=0)
Biopython version 1.77


## Chapter 1 Databases and File Formats
The use of bioinformatics typically involves dealing with vast amounts of biological data. This can range from DNA sequences, protein sequences, protein structures, and all their associated annotations. This information is stored into various file formats and can be manipulated and read using biopython. Each file format has unique characteristics and information that a biologist might want to manipulate. In the interest of being able to access and manipulate this data, one must be able to understand these file formats, and where to access this information from. 

### Databases
Biological data, ranging from gene sequences to protein sequences is stored in databases. 

Beginning with DNA, there are **three** main databases commonly used when dealing with nucleotide sequences: 

1. GenBank, provided by the National Center for Biotechnology Information (NCBI)
2. European Molecular Biology Laboratory (EMBL)-Bank, provided by the European Bioinformatics Institute (EBI)
3. DNA Database of Japan (DDBJ), provided by the National Institute of Genetics in Mishima

These databases are coordinated by the International Nucleotide Sequence Database Collaboration 
(INSDC). Each of these databases are but a part of the resources and other database of interest provided by the NCBI, EBI, and DDBJ.

### File Formats
As mentioned, databases store a wide range of relevent biological information. Some of this information may include large DNA sequences, mRNA products, or proteins sequences. These sequences alone are fairly simple linear arangements of nucleotides and amino acids. However, the context around these sequences, such as the name of the organism, chromosome, gene, or intron they are taken from is also important. 

In order to keep track of this information such that its kept organized and a biologist can understand it, several standardized file formats have been developed, a list of which can be found [here](https://www.algosome.com/articles/bioinformatics-sequence-file-formats.html). Some of these include:

* GenBank, which can store a wide variety of sequence information
* EMBL, similar to GenBank, but refit for the EBI database
* ABI, which stores pure sequence information in binary. usually only used in special cases.
* PDB, protein database formate, used to store sequence information, and the protein's 3D structure gained through crystalography
* MLD, mostly stores information regarding smaller molecular strucutres, very similar to PDB
* BAM/SAM, stores next generation sequencing data, containing both a binary sequence readout and an alphabetical sequence readout.
* SFF, also stores next generation sequencing information, specifically the sequencing information from Ion-Torrent and Roche's '454'.

Each database uses their own unique type of format, however they each have similarities between them. This document covers some of the more commonly used file types typically encountered by bioinformatics students.

#### FASTA File Format
FASTA is one of the simplist of the sequence data formats. FASTA usually contains some idintifying information for the sequence of interest in a header, followed either by the actual DNA or protein sequence. An official description of the file format can be found [here](https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=BlastHelp).

##### FASTA files look something like this:

Fasta files are quite simple to read:
* Generally, Fasta files begin with an information line, usually denoted with an ">" symbole.
* Stored sequence pertaining to that information line follows.


#### GenBank File Format
GenBank, EMBL, and DDBJ offer a wide range of molecular sequence data. We will use GenBank to demonstrate how this sequence information is organized and how to access it.GenBank mainly deals with genomic DNA sequences, their theoretically transcribed pre-mRNA, mRNA products, and their translated protein sequences. GenBank provides this and more information regarding specific proteins sequences through the GenBank file format.

GenBank uses a standardized file format to contain pertinent information for any given DNA sequence. The GenBank format helps keep information pertaining to a particular gene or protein relatively standard across platforms and help with ease of access. An offical description of each element in the format can be found [here](https://www.ncbi.nlm.nih.gov/Sitemap/samplerecord.html) at the NCBI's website.

##### GenBank files look something like this:

GenBank files appear vastly more complicated than FASTA files at an initial glance. However, FASTA files and GenBank files function quite similarly:
* instead of a simple identification line denoted by the ">" like in a FASTA file, Gene bank files can contain a fast amount of intial information pertaining to the gene of interest at the start of the file.
* GenBank files can contain many features related to specific sequences within the file.
* like FASTA files, GenBank files also contain the sequence information after the initial header.

## Chapter 5 Substitution Matrices
One of the fundamental operations in bioinformatics is comparing two sequences, either nucleic acids or proteins, and line them up to archieve maximal levels of identity. This is called _**pairwise sequence alignment**_. Quantification of the similarity of the two sequences is important for establishing whether they are homologs. There are two widely used methods developed to quantify the similarity.

- Method 1: align closely related homologs and count the frequencies of amino acid substitutions.
- Method 2: use a database of aligned sequences derived from protein domains that have a particular structure or function. The frequencies of amino acid substitutions are recorded.

These two methods gave rise to the *PAM* and *BLOSUM* series of amino acid substitution matrices, respectively. These substitution matrices are used in many sequence alignment tools.

The *Biopython* package includes these substitution matrices.

In [2]:
# this scriptlet display BLOSUM62 and PAM250 matrices
from Bio.Align import substitution_matrices as smatrices
blosum62 = smatrices.load("BLOSUM62")
pam250 = smatrices.load("PAM250")
print(blosum62)
print("-"*80)
print(pam250)

#  Matrix made by matblas from blosum62.iij
#  * column uses minimum score
#  BLOSUM Clustered Scoring Matrix in 1/2 Bit Units
#  Blocks Database = /data/blocks_5.0/blocks.dat
#  Cluster Percentage: >= 62
#  Entropy =   0.6979, Expected =  -0.5209
     A    R    N    D    C    Q    E    G    H    I    L    K    M    F    P    S    T    W    Y    V    B    Z    X    *
A  4.0 -1.0 -2.0 -2.0  0.0 -1.0 -1.0  0.0 -2.0 -1.0 -1.0 -1.0 -1.0 -2.0 -1.0  1.0  0.0 -3.0 -2.0  0.0 -2.0 -1.0  0.0 -4.0
R -1.0  5.0  0.0 -2.0 -3.0  1.0  0.0 -2.0  0.0 -3.0 -2.0  2.0 -1.0 -3.0 -2.0 -1.0 -1.0 -3.0 -2.0 -3.0 -1.0  0.0 -1.0 -4.0
N -2.0  0.0  6.0  1.0 -3.0  0.0  0.0  0.0  1.0 -3.0 -3.0  0.0 -2.0 -3.0 -2.0  1.0  0.0 -4.0 -2.0 -3.0  3.0  0.0 -1.0 -4.0
D -2.0 -2.0  1.0  6.0 -3.0  0.0  2.0 -1.0 -1.0 -3.0 -4.0 -1.0 -3.0 -3.0 -1.0  0.0 -1.0 -4.0 -3.0 -3.0  4.0  1.0 -1.0 -4.0
C  0.0 -3.0 -3.0 -3.0  9.0 -3.0 -4.0 -3.0 -3.0 -1.0 -1.0 -3.0 -1.0 -2.0 -3.0 -1.0 -1.0 -2.0 -2.0 -1.0 -3.0 -3.0 -2.0 -4.0
Q -1.0  1.0  0.0  0.

## Chapter 7 Molecular Graphics

In this section, we use NGLView to display a structure of interest.


In [ ]:
#In this block, the user will be prompted to enter the name of the pdb code that they would like to look at.
text_box = str(input("Please enter the PDB code of the structure: "))

In [ ]:
# we minic the folder structure of PDB database and save the pdf file in the corresponding folder.
first_pdb_file = PDBList()
name = first_pdb_file.retrieve_pdb_file(text_box)
protein_file = ""
last_value = -1
print(name[-2])

if (platform.system() == 'Windows'):
    while (name[last_value] != '\\'):
        protein_file += name[last_value] #Each of the letters is added to the protein_name string, starting from the last letter.
        #The value of last_value (which is supposed to immitate the index) is reduced by one, and since it's negative, the constantly decreasing value goes towards the beginning of the string.
        last_value -= 1 

else:
    while (name[last_value] != '/'):
        protein_file += name[last_value] #Each of the letters is added to the protein_name string, starting from the last letter.
        #The value of last_value (which is supposed to immitate the index) is reduced by one, and since it's negative, the constantly decreasing value goes towards the beginning of the string.
        last_value -= 1 

protein_file = protein_file[::-1] #The protein name is reversed, to get a proper pdb file format.

protein_name = ""

index = 0
while(protein_file[index] != '.'):
    protein_name += protein_file[index]
    index += 1

protein_name = protein_name.upper()
print("The name of the protein is, " + protein_file)

In [ ]:
protein_class = str(protein_file[1]) + str(protein_file[2])

#We create an instance of the MMCIF Parser, to load the protein file.
parser = MMCIFParser()

path = os.path.join(protein_class, protein_file)
structure = parser.get_structure(protein_name, path)
path = os.path.join(protein_class, protein_file)
structure = parser.get_structure(protein_name, path)

In [ ]:

def clean_protein(obj):
    print(', '.join([a for a in dir(obj) if not a.startswith('_')]))
    
clean_protein(structure)

In [ ]:
view_one = nv.show_biopython(structure)
view_one

In [ ]:
view_two = nv.show_biopython(structure)
view_two.add_ball_and_stick()
view_two

In [ ]:
# clean up the folder if neccessary
os.remove(path) #Removes the file after the user is done looking at the protein.
os.rmdir(protein_class)